In [ ]:
# alerter.py v1.5
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import requests
import traceback

# ==============================================================================
# --- БЛОК 1: КОНФИГУРАЦИЯ И ПОДКЛЮЧЕНИЕ ---
# ==============================================================================
CREDS_FILE = 'credentials.json'
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1qBYS_DhGNsTo-Dnph3g_H27aHQOoY0EOcmCIKarb7Zc/"
SCOPE = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive.file']

def get_worksheet(sheet_name):
    try:
        creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
        client = gspread.authorize(creds)
        spreadsheet = client.open_by_url(SPREADSHEET_URL)
        return spreadsheet.worksheet(sheet_name)
    except Exception as e:
        print(f"❌ Ошибка доступа к листу '{sheet_name}': {e}"); return None

# ==============================================================================
# --- БЛОК 2: ЛОГИКА ОТПРАВКИ АЛЕРТОВ ---
# ==============================================================================

### НОВОЕ: ФУНКЦИЯ ДЛЯ ЭКРАНИРОВАНИЯ СИМВОЛОВ MARKDOWN ###
def escape_markdown(text: str) -> str:
    """Экранирует специальные символы MarkdownV2 для безопасной отправки в Telegram."""
    escape_chars = r'_*[]()~`>#+-=|{}.!'
    return ''.join(f'\\{char}' if char in escape_chars else char for char in text)

def send_telegram_alert(bot_token, chat_id, message):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    params = {'chat_id': chat_id, 'text': message, 'parse_mode': 'MarkdownV2'}
    try:
        response = requests.post(url, json=params) # Рекомендуется использовать POST для отправки данных
        response.raise_for_status()
        print(f"    >> ✅ Алерт успешно отправлен в Telegram!")
        return True
    except requests.exceptions.HTTPError as e:
        print(f"    >> ❌ Ошибка отправки алерта в Telegram: {e}")
        print(f"    >> ❌ Ответ сервера: {e.response.text}") # Выводим текст ошибки от Telegram
        return False

# ==============================================================================
# --- БЛОК 3: ГЛАВНАЯ ЛОГИКА АЛЕРТЕРА ---
# ==============================================================================
def main_alerter():
    print("\n" + "="*50)
    print("--- 🔔 АСУП ИИ: Система Оповещений v1.5 (Отлаженная) 🔔 ---")
    print("="*50)

    config_sheet = get_worksheet('Config')
    analysis_sheet = get_worksheet('Analysis')
    if not all([config_sheet, analysis_sheet]): return

    print("🔄 Читаю настройки и данные для анализа...")
    configs_raw = config_sheet.get_all_records()
    configs = {item['Parameter']: item['Value'] for item in configs_raw}

    analysis_data = analysis_sheet.get_all_records(value_render_option='UNFORMATTED_VALUE')
    if len(analysis_data) < 2:
        print("ℹ️ Лист 'Analysis' пуст или содержит только заголовок. Пропускаю.")
        return

    analysis_df = pd.DataFrame(analysis_data[1:], columns=analysis_data[0]) # Используем первую строку как заголовки

    bot_token = configs.get('TELEGRAM_BOT_TOKEN')
    chat_id = configs.get('TELEGRAM_CHAT_ID')

    try:
        rsi_level = float(configs.get('RSI_ALERT_LEVEL', 30))
    except (ValueError, TypeError):
        print("❌ КРИТИЧЕСКАЯ ОШИБКА: RSI_ALERT_LEVEL в 'Config' не является числом."); return

    if not all([bot_token, chat_id]):
        print("❌ КРИТИЧЕСКАЯ ОШИБКА: TELEGRAM_BOT_TOKEN или TELEGRAM_CHAT_ID не найдены."); return

    print(f"⚙️ Проверяю условия. Пороговый RSI: {rsi_level}")

    alerts_sent_count = 0
    for index, row in analysis_df.iterrows():
        ticker = row.get('Ticker')
        rsi_value = row.get('RSI_14')

        if not isinstance(rsi_value, (int, float)): continue
        alert_sent = str(row.get('Alert_Sent')).upper() == 'TRUE'

        if rsi_value < rsi_level and not alert_sent:
            print(f"  - ❗️ НАЙДЕН СИГНАЛ для {ticker}! RSI: {rsi_value} < {rsi_level}")

            ### ИЗМЕНЕНИЕ: ЭКРАНИРУЕМ ТИКЕР ПЕРЕД ОТПРАВКОЙ ###
            safe_ticker = escape_markdown(ticker)

            message = (
                f"🚨 *СИГНАЛ: ПЕРЕПРОДАННОСТЬ*\n\n"
                f"*{safe_ticker}* вошел в зону перепроданности\n\n"
                f"*Текущий RSI\\(14\\):* `{rsi_value:.2f}` \\(Ниже порога `{rsi_level}`\\)\n\n"
                f"*Трактовка:* Актив сильно упал, продавцы теряют силу\\. Вероятность технического отскока вверх повышена\\.\n\n"
                f"*Рекомендация:* Искать точку для покупки\\. Проверить новости\\. Искать разворотные свечные модели на графике\\."
            )

            if send_telegram_alert(bot_token, chat_id, message):
                alerts_sent_count += 1
                try:
                    # Находим столбец по имени, а не по жестко заданному индексу
                    headers = analysis_sheet.row_values(1)
                    alert_sent_col_index = headers.index('Alert_Sent') + 1
                    analysis_sheet.update_cell(index + 2, alert_sent_col_index, 'TRUE')
                    print(f"  - Установлен флаг Alert_Sent для {ticker}.")
                except ValueError:
                    print(" - ⚠️ Не найден столбец 'Alert_Sent' для обновления флага.")
                except Exception as e:
                    print(f" - ⚠️ Ошибка при обновлении флага Alert_Sent: {e}")

    if alerts_sent_count == 0:
        print("✅ Новых сигналов, требующих оповещения, не найдено.")

    print("--- 🏁 РАБОТА СИСТЕМЫ ОПОВЕЩЕНИЙ ЗАВЕРШЕНА 🏁 ---")

if __name__ == "__main__":
    main_alerter()